In [65]:
import os, sys

# 添加包路径到sys.path
package_path = '/Users/yahoo168/Desktop/資料庫_測試功能/alphahelix-database-cloud'
if package_path not in sys.path:
    sys.path.append(package_path)

# 现在可以导入包中的模块
# import your_module

from external_tools import GoogleCloudStorageTools #type: ignore
from utils import * #type: ignore
from datetime import datetime, timedelta

google_cloud_storage_tools = GoogleCloudStorageTools(credential_file_path="/Users/yahoo168/Desktop/GOOGLE_APPLICATION_CREDENTIALS.json")


In [73]:
def _organize_blob_to_document_meta(blob_list, start_date, end_date, document_type):
    document_meta_list = list()
    for blob in blob_list:
        # 取得資料的上傳時間戳記（upload_timestamp)
        upload_timestamp = datetime.fromtimestamp(int(blob.metadata["upload_timestamp"]))
        # 若未載明資料時間(data_timestamp)，則以上傳時間替代
        if "data_timestamp" in blob.metadata:
            data_timestamp = datetime.fromtimestamp(int(blob.metadata["data_timestamp"]))
        else:
            data_timestamp = upload_timestamp
        
        # 检查开始日期和结束日期
        if (start_date is not None and upload_timestamp < start_date) or \
           (end_date is not None and upload_timestamp > end_date):
            continue
            
        document_meta_list.append({"upload_timestamp": upload_timestamp,
                                   "data_timestamp": data_timestamp,
                                    "source": blob.metadata["source"],
                                    #去除folder路徑和副檔名，用於頁面呈現（最多70個字，避免影響頁面）
                                    "file_name": blob.name.split("/")[-1].split(".")[0][:70], 
                                    "url": blob.public_url,
                                    "blob_name": blob.name,
                                    "document_type": document_type})
    return document_meta_list

# 搜尋特定市場（TW/US）的股票文件（memo或report）
def search_investment_gcs_document(country, document_type, equity_type="stock", ticker=None, start_date=None, end_date=None):
    # 如果有ticker，則搜尋特定ticker的文件，否則搜尋所有文件
    if ticker:
        folder_name = f"{country}_{equity_type}_{document_type}/{ticker}"
    else:
        folder_name = f"{country}_{equity_type}_{document_type}/"
    
    blob_list = google_cloud_storage_tools.get_blob_list_in_folder(bucket_name="investment_report", folder_name=folder_name)
    document_meta_list = _organize_blob_to_document_meta(blob_list, start_date, end_date, document_type)
    return document_meta_list

# 搜尋近期的GCS document（以data timestamp搜尋為基礎）
def search_recent_gcs_document(days=1):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    folder_name_list = ["TW_industry_report/", "TW_stock_memo/", "TW_stock_report/", "US_stock_report/"]
    document_meta_list = list()
    for folder_name in folder_name_list:
        document_type = folder_name.split("_")[-1]
        _part_blob_list = google_cloud_storage_tools.get_blob_list_in_folder(bucket_name="investment_report", folder_name=folder_name)
        _part_document_meta_list = _organize_blob_to_document_meta(_part_blob_list, start_date, end_date, document_type)
        document_meta_list.extend(_part_document_meta_list)

    return document_meta_list

In [75]:
document_meta_list = search_recent_gcs_document(days=3)

In [1]:
len("1719551523_2308台達電 ML 20240628")

30